# Machine Learning Model

### Import Dependencies

In [1]:
import pandas as pd
from pathlib import Path
from census import Census
from us import states

# Ignnore warning
import warnings
warnings.simplefilter(action='ignore')

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean


from config import api_census_key

## Import data from CSV
This data will be used to train the BlackBox model

In [2]:
csv_in = Path('../Model/model_blackbox_shooters_v8.csv')
shooter_profile = pd.read_csv(csv_in)

In [4]:
shooter_profile.head()

,Age,Gender,Race,Immigrant,Education,RelStatus,Employed,Work,MilService,Arrested,ParentDivorce,SES,MentalIllness,MentalIllnessHistory,Autism,HealthIssues,Classification
0,25,Male,White,No,Some college/trade school,Married,Not working,Unknown,Yes,Yes,No evidence,Middle class,Yes,No evidence,No evidence,Yes,1
1,18,Male,White,No,Less than high school,Single,Not working,Unknown,No,No,No evidence,Middle class,Yes,No evidence,No evidence,No evidence,1
2,39,Male,White,No,Some college/trade school,Married,Working,In between,Yes,No,No evidence,Middle class,No evidence,No evidence,No evidence,No evidence,1
3,56,Male,White,No,Unknown,Single,Working,Blue collar,No,Yes,No evidence,Unknown,No evidence,No evidence,No evidence,No evidence,1
4,31,Male,Black,No,Some college/trade school,Married,Not working,In between,Yes,No,Yes,Middle class,Yes,No evidence,No evidence,No evidence,1


## Save table in SQL
- Table `shooters`: contains all the shooter data (same as the CSV above)
- Table `analysis`: will contain data that can be loaded and analysed from the Web App

In [10]:
# Get Base
Base = declarative_base()

# Create Shooter class
class Shooter(Base):
    __tablename__ = 'shooters'
    shooter_id = Column(Integer, primary_key=True)
    Age = Column(Integer)
    Gender = Column(String)
    Race = Column(String)
    Immigrant = Column(String)
    Education = Column(String)
    RelStatus = Column(String)
    Employed = Column(String)
    Work = Column(String)
    MilService = Column(String)
    Arrested = Column(String)
    ParentDivorce = Column(String)
    SES = Column(String)
    MentalIllness = Column(String)
    MentalIllnessHistory = Column(String)
    Autism = Column(String)
    HealthIssues = Column(String)
    Classification = Column(Integer)
    Probability = Column(Float)

# Create Analysis class (used to load data to classify)
class Analysis(Base):
    __tablename__ = 'analysis'
    shooter_id = Column(Integer, primary_key=True)
    Age = Column(Integer)
    Gender = Column(String)
    Race = Column(String)
    Immigrant = Column(String)
    Education = Column(String)
    RelStatus = Column(String)
    Employed = Column(String)
    Work = Column(String)
    MilService = Column(String)
    Arrested = Column(String)
    ParentDivorce = Column(String)
    SES = Column(String)
    MentalIllness = Column(String)
    MentalIllnessHistory = Column(String)
    Autism = Column(String)
    HealthIssues = Column(String)
    Classification = Column(Integer)
    Probability = Column(Float)
    

# Create a connection to a SQLite database
engine = create_engine('sqlite:///../Server/blackbox_db.sqlite')

# Create the tables within the database
Base.metadata.create_all(engine)

# Start session
session = Session(bind=engine)

In [11]:
# Loop through risk DataFrame and retrieve data
for index, row in shooter_profile.iterrows():
    # Add data to database
    session.add(Shooter(
        Age = int(shooter_profile.loc[index,'Age']),
        Gender = shooter_profile.loc[index,'Gender'],
        Race = shooter_profile.loc[index,'Race'],
        Immigrant = shooter_profile.loc[index,'Immigrant'],
        Education = shooter_profile.loc[index,'Education'],
        RelStatus = shooter_profile.loc[index,'RelStatus'],
        Employed = shooter_profile.loc[index,'Employed'],
        Work = shooter_profile.loc[index,'Work'],
        MilService = shooter_profile.loc[index,'MilService'],
        Arrested = shooter_profile.loc[index,'Arrested'],
        ParentDivorce = shooter_profile.loc[index,'ParentDivorce'],
        SES = shooter_profile.loc[index,'SES'],
        MentalIllness = shooter_profile.loc[index,'MentalIllness'],
        MentalIllnessHistory = shooter_profile.loc[index,'MentalIllnessHistory'],
        Autism = shooter_profile.loc[index,'Autism'],
        HealthIssues = shooter_profile.loc[index,'HealthIssues'],
        Classification = 1,
        Probability = 100.0
    ))

print(f"{len(shooter_profile)} rows ready for commit.")

193 rows ready for commit.


In [12]:
# Commit changes to session
session.commit()

In [13]:
# Close session
session.close()